In [1]:
# Considering this name for the package. Doesn't seem to be currently taken in PyPI
# import aimodels

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

# Thinking this and the `Provider Interface should move to the
# `framework` directory
class ChatCompletionResponse:
    class Choice:
        class Message:
            def __init__(self):
                self.content = None

        def __init__(self):
            self.message = ChatCompletionResponse.Choice.Message()

    def __init__(self):
        self.choices = [ChatCompletionResponse.Choice()]  # Adjust the range as needed for more choices

class ProviderInterface():
    def chat_completion_create(self, messages=None, model=None, temperature=0):
        raise NotImplementedError("Provider Interface has not implemented chat_completion_create()") 

# These should go in the `providers` package
class OpenAIInterface(ProviderInterface):
    def __init__(self):
        import openai 
        self.openai_client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

    def chat_completion_create(self, messages=None, model=None, temperature=0):
        return self.openai_client.chat.completions.create(model=model, temperature=temperature, messages=messages)
    
    
class AnthropicInterface(ProviderInterface):
    def __init__(self):
        import anthropic
        self.anthropic_client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

    def chat_completion_create(self, messages=None, model=None, temperature=0):
        anthropic_messages = []
        system_message = None 
        for msg in messages:
            if "role" in msg and "content" in msg:
                if msg["role"] == "system":
                    system_message = msg["content"]
                    #print(f"Found system message: {system_message}")
                else:
                    temp_msg = msg.copy()
                    temp_msg["content"] = [{"type": "text","text": temp_msg["content"] }]
                    anthropic_messages.append(temp_msg)
        #print(f"Messages: {messages}\nAnthropic messages: {anthropic_messages}")
    
        if system_message is None:
            response = self.anthropic_client.messages.create(messages=anthropic_messages, model=model, max_tokens = 4096, temperature=temperature)
        else: 
            response = self.anthropic_client.messages.create(messages=anthropic_messages, model=model, system=system_message, max_tokens = 4096, temperature=temperature)

        text_response = response.content[0].text

        chat_completion_response = ChatCompletionResponse()
        chat_completion_response.choices[0].message.content = text_response

        return chat_completion_response 

class GroqInterface(ProviderInterface):
    def __init__(self):
        import groq
        self.groq_client = groq.Groq(api_key=os.getenv("GROQ_API_KEY"))

    def chat_completion_create(self, messages=None, model=None, temperature=0):
        return self.groq_client.chat.completions.create(model=model, temperature=temperature, messages=messages)

class AWSInterface(ProviderInterface):
    def __init__(self):
        import boto3
        import json
        self.aws_client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")
        self.json = json
        
    def chat_completion_create(self, messages=None, model=None, temperature=0):
        anthropic_messages = []
        system_message = None 
        for msg in messages:
            if "role" in msg and "content" in msg:
                if msg["role"] == "system":
                    system_message = msg["content"]
                else:
                    temp_msg = msg.copy()
                    temp_msg["content"] = [{"type": "text","text": temp_msg["content"] }]
                    anthropic_messages.append(temp_msg)
        #print(f"Messages: {messages}\nAnthropic messages: {anthropic_messages}")
    
        if system_message is None:
            response = self.aws_client.invoke_model(
                modelId=model,
                body=self.json.dumps({"anthropic_version": "bedrock-2023-05-31", "max_tokens": 4096, "messages": anthropic_messages}),
            )
        else: 
            response = self.aws_client.invoke_model(
                modelId=model, 
                body=self.json.dumps({"anthropic_version": "bedrock-2023-05-31", "max_tokens": 4096, "system":system_message, "messages": anthropic_messages}),
            )
        
        temp = self.json.loads(response.get("body").read()) 
        output_list = temp.get("content", [])
        temp2 = output_list[0].get("text")

        chat_completion_response = ChatCompletionResponse()
        chat_completion_response.choices[0].message.content = temp2
        return chat_completion_response 

        #input_tokens = result["usage"]["input_tokens"]
        #output_tokens = result["usage"]["output_tokens"]


class OllamaInterface(ProviderInterface):
    def __init__(self):
        import ollama
        self.ollama = ollama

    def chat_completion_create(self, messages=None, model=None, temperature=0):
        response = self.ollama.chat(model=model, messages=messages)
        text_response = response['message']['content']
        chat_completion_response = ChatCompletionResponse()
        chat_completion_response.choices[0].message.content = text_response

        return chat_completion_response 



In [2]:
# should we make a `client` package for these?
class MultiFMClient:
    def __init__(self):
        self.chat = self.Chat(self)
        self.num=10
        self.all_interfaces = {} 
        self.all_factories = {"openai":OpenAIInterface, 
                              "groq":GroqInterface,
                              "anthropic":AnthropicInterface,
                              "aws":AWSInterface,
                              "ollama":OllamaInterface} 

    def get_provider_interface(self, model):
        # Given a model name like "openai:gpt-4" split it into provider="openai" and model_name="gpt-4" 
        # We split on the first :, so "ollama:llama3:8b" gets split into "ollama" and "llama3:8b" 
        if not ":" in model:
            raise ValueError(f"Expected : in model identifier to specify provider:model. Got {model}")
        model_parts = model.split(":", maxsplit=1)
        provider = model_parts[0]
        model_name = model_parts[1]
        # print(f"Provider: {provider}, modelname: {model_name}")
        
        if provider in self.all_interfaces:
            #print(f("Found provider {provider} in all_interfaces")) 
            return self.all_interfaces[provider] 

        if not provider in self.all_factories:
            raise Exception(f"Could not find factory to create interface for provider {provider}") 

        self.all_interfaces[provider] = self.all_factories[provider]() 

        return (self.all_interfaces[provider], model_name)
    
    class Chat:
        def __init__(self, topmost_instance):
            self.topmost_instance = topmost_instance 
            self.completions = self.Completions(topmost_instance)


        class Completions:
            def __init__(self, topmost_instance):
                self.topmost_instance = topmost_instance 

            def create(self, model=None, temperature=0, messages=None):
                (interface, modelname) = self.topmost_instance.get_provider_interface(model)
                temp = interface.chat_completion_create(messages, model=modelname)
                return temp

# Usage example
client = MultiFMClient()
messages=[{"role": "system", "content": "Respond in Pirate English."}, 
          {"role": "user", "content": "Tell me a joke"} ] 

models_list = ["anthropic:claude-3-opus-20240229", "aws:anthropic.claude-3-sonnet-20240229-v1:0", "openai:gpt-4-turbo", "groq:mixtral-8x7b-32768", ] 

for model_id in models_list:
    print(f"----------\nResults from {model_id}: \n")
    response = client.chat.completions.create(model=model_id, messages=messages)
    print(response.choices[0].message.content) 


----------
Results from anthropic:claude-3-opus-20240229: 

Arrr, me bucko, 'ere be a jolly jest fer ye!

What did th' pirate say on 'is 80th birthday? "Aye matey!"

Ye see, it be a play on words, as "Aye matey" sounds like "I'm eighty". Har har har! 'Tis a clever bit o' pirate humor, if I do say so meself. Now, 'ow about ye fetch me a mug o' grog while I spin ye another yarn?
----------
Results from aws:anthropic.claude-3-sonnet-20240229-v1:0: 

Ahoy, matey! 'Tis a fine day fer a jolly joke, ain't it? Here be one fer ye:

Why did th' pirate buy a trampoline? 'Cause he wanted t' bounce off th' ocean waves! Arr arr arr!
----------
Results from openai:gpt-4-turbo: 



KeyboardInterrupt: 

In [47]:
print(temp)

None


In [1]:
class MultiFM:
    def __init__(self):
        self.temp = 5
    def set_temp(self, newtemp):
        self.temp = newtemp
    def returntemp(self):
        return self.temp

# Older version 

In [70]:
import openai
import os 
from groq import Groq
import anthropic

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai_client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
anthropic_client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


def get_response_to_prompt(prompt,model="openai:gpt-4-turbo"):
    model_parts = model.split(":")
    fm_provider = model_parts[0]
    model = model_parts[1]

    match fm_provider:
        case "openai":
            response = openai_client.chat.completions.create(model=model, temperature=0, messages=[
                # {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt} ] )
            return response.choices[0].message.content 

        case "anthropic":
            response = anthropic_client.messages.create(model=model, max_tokens=1000, temperature=0,
                messages=[{"role": "user","content": [
                    {"type": "text","text": prompt }] }] )
            return response.content[0].text

        case "groq": 
            response = groq_client.chat.completions.create(
                messages=[{"role": "user","content": prompt}], model=model)
            return response.choices[0].message.content
        case _:
            raise Exception(f"FM Provider {provider} not found")

            






In [71]:
prompt = "What's the gravity on Jupiter?"

temp1 = get_response_to_prompt(prompt,model="openai:gpt-4-turbo")
print(temp1)
temp2 = get_response_to_prompt(prompt,model="anthropic:claude-3-opus-20240229")

print(temp2)
temp3 = get_response_to_prompt(prompt,model="groq:mixtral-8x7b-32768")
print(temp3) 


The gravity on Jupiter is significantly stronger than on Earth. On Jupiter, the gravitational force is about 24.79 m/s², which is approximately 2.5 times stronger than the gravitational force on Earth, which is about 9.8 m/s². This means that if you could stand on Jupiter (which is impossible due to its lack of a solid surface and extreme atmospheric conditions), you would feel much heavier.
The gravity on Jupiter varies depending on the distance from the center of the planet, as Jupiter is a gas giant and does not have a solid surface like Earth. However, we can calculate the gravity at the "surface" of Jupiter, which is typically defined as the point where the atmospheric pressure is equal to 1 bar (similar to Earth's atmospheric pressure at sea level).

At Jupiter's "surface," the acceleration due to gravity is approximately 24.79 m/s² (meters per second squared). This is about 2.53 times the gravity on Earth's surface, which is 9.81 m/s².

To put this into perspective, if you weigh

In [64]:
print(temp2[0].text)

The gravity on Jupiter varies depending on the distance from the center of the planet, as Jupiter is a gas giant and does not have a solid surface like Earth. However, we can calculate the gravity at the "surface" of Jupiter, which is typically defined as the point where the atmospheric pressure is equal to 1 bar (similar to Earth's atmospheric pressure at sea level).

At Jupiter's "surface," the acceleration due to gravity is approximately 24.79 m/s² (meters per second squared). This is about 2.53 times the gravity on Earth's surface, which is 9.81 m/s².

To put this into perspective, if you weighed 150 pounds (68 kg) on Earth, you would weigh about 379.5 pounds (172 kg) on Jupiter's "surface." However, since Jupiter is a gas giant, you would not be able to stand on its surface as you would sink into its atmosphere.


In [73]:
print(get_response_to_prompt(prompt,model="groq:llama3-70b-8192"))

The surface gravity of Jupiter is approximately 24.79 meters per second squared (m/s²), which is about 2.5 times the surface gravity of Earth, which is about 9.8 m/s².

This means that if you weighed 100 pounds (45 kg) on Earth, you would weigh around 250 pounds (113 kg) on Jupiter!

It's worth noting that Jupiter is a gas giant planet, so it doesn't really have a solid surface. The gravity measurement I mentioned is based on the gravitational acceleration at a specific altitude, called the "1-bar level," which is defined as the point where the atmospheric pressure is equal to 1 bar (about 1 atmosphere).


In [22]:

def raw_response_to_prompt(prompt,model="gpt-4-turbo"):
    return client.chat.completions.create(model=model, temperature=0, messages=[
            # {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )



In [9]:
response = get_response_to_prompt("""
Classify the following as having either a POSITIVE or NEGATIVE sentiment:

---
The HP meeting was great! Lots of exciting progress and discussios on AI. 
---
""")

print(response)


POSITIVE


In [51]:
response = raw_response_to_prompt("""
Classify the following as having either a POSITIVE or NEGATIVE sentiment:

---
The HP meeting was great! Lots of exciting progress and discussios on AI. 
---
""")

print(response)


ChatCompletion(id='chatcmpl-9JtlARfwqtAEoxpnp65EwOpjO7HVe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='POSITIVE', role='assistant', function_call=None, tool_calls=None))], created=1714529600, model='gpt-4-turbo-2024-04-09', object='chat.completion', system_fingerprint='fp_46a93fa712', usage=CompletionUsage(completion_tokens=2, prompt_tokens=42, total_tokens=44))


In [53]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

In [58]:
error("FdsF")

NameError: name 'error' is not defined

In [10]:
import anthropic


In [16]:

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key= os.environ.get("ANTHROPIC_API_KEY"),
)
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": "What is gravity on Mars? "
        }
    ]
)
print(message.content)

[ContentBlock(text="The gravity on Mars is approximately 3.71 m/s², which is about 38% of the gravity on Earth (9.81 m/s²). This means that an object on Mars would weigh about 38% of its weight on Earth.\n\nThe lower gravity on Mars is due to its smaller mass compared to Earth. Mars has a mass of about 6.39 × 10^23 kg, which is roughly 10.7% of Earth's mass (5.97 × 10^24 kg).\n\nThe lower gravity on Mars has implications for future human exploration and potential colonization. Astronauts on Mars would experience reduced physical stress on their bodies, but they would also need to adapt to the differences in gravity for daily activities and movement. The lower gravity also affects the planet's ability to retain atmosphere, which is one of the reasons why Mars has a much thinner atmosphere compared to Earth.", type='text')]


In [15]:
print(message.content[0].text)

The gravity on Mars is approximately 3.71 m/s², which is about 38% of the gravity on Earth (9.81 m/s²). This means that an object on Mars would weigh about 38% of its weight on Earth.

The lower gravity on Mars is due to its smaller mass compared to Earth. Mars has a mass of about 6.39 × 10^23 kg, which is roughly 10.7% of Earth's mass (5.97 × 10^24 kg).

The lower gravity on Mars has implications for future human exploration and potential colonization. Astronauts on Mars would experience reduced physical stress on their bodies, but they would also need to adapt to the differences in gravity for tasks like walking, running, and lifting objects. The lower gravity also affects the planet's ability to retain atmosphere, which is one of the reasons why Mars has a much thinner atmosphere compared to Earth.


In [ ]:

client = myclient.client()
client.chat.completions.create(17)



class FMClient:
    
    model_parts = model.split(":")
    fm_provider = model_parts[0]
    model = model_parts[1] 

    match fm_provider:
        case "openai":
            response = openai_client.chat.completions.create(model=model, temperature=0, messages=[
                # {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt} ] )
            return response.choices[0].message.content 

        case "anthropic":
            response = anthropic_client.messages.create(model=model, max_tokens=1000, temperature=0,
                messages=[{"role": "user","content": [
                    {"type": "text","text": prompt }] }] )
            return response.content[0].text

        case "groq": 
            response = groq_client.chat.completions.create(
                messages=[{"role": "user","content": prompt}], model=model)
            return response.choices[0].message.content
        case _:
            raise Exception(f"FM Provider {provider} not found")

            
client = openai.OpenAI()


def get_response_to_prompt(prompt,model="gpt-4-turbo"):
    return client.chat.completions.create(model=model, temperature=0, messages=[
            # {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    ).choices[0].message.content
response = get_response_to_prompt("Tell me a joke.")

print(response)


# Anthropic 

In [ ]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key= os.environ.get("ANTHROPIC_API_KEY"),
)
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is gravity on Mars? "
                }
            ]
        }
    ]
)
print(message.content)

# Groq


In [68]:
import os

from groq import Groq

client = Groq(api_key=os.environ.get("GROQ_API_KEY"),)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Tell me a joke about cats",
        }
    ],
    model="mixtral-8x7b-32768",
)



In [69]:
print(chat_completion.choices[0].message.content)


What do you call a pile of kittens? A meow-ntain! I'll show myself out...


# Ollama

In [ ]:
from langchain.llms import Ollama


In [ ]:
import langchain

In [ ]:
ollama = Ollama(base_url='http://localhost:11434',model="llama2")


In [149]:
import ollama
response = ollama.chat(model='llama3:8b', messages=[
  {
    'role': 'user',
    'content': 'Tell me a joke.',
  },
])
print(response['message']['content'] )

Here's one:

Why don't scientists trust atoms?

Because they make up everything!

Hope that made you smile! Do you want to hear another?


"\nSure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing!"

In [146]:
print(x)

{'model': 'llama2', 'created_at': '2024-05-03T00:15:15.95371Z', 'message': {'role': 'assistant', 'content': "\nSure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing!"}, 'done': True, 'total_duration': 787894500, 'load_duration': 2937084, 'prompt_eval_duration': 200606000, 'eval_count': 39, 'eval_duration': 581242000}



# PaLM


In [ ]:
import google.generativeai as palm
from google.api_core import client_options as client_options_lib


In [ ]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [ ]:
palm.configure(
    api_key=GOOGLE_API_KEY,
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

In [ ]:
models = [m for m in palm.list_models() 
          if 'generateText' 
          in m.supported_generation_methods]
models

In [ ]:
model_bison = models[3]
model_bison

In [ ]:
from google.api_core import retry
@retry.Retry()
def generate_text(prompt,
                  model=model_bison,
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)

In [ ]:
prompt = """You are an expert in social services, poverty, abuse, and disadvantaged populations.  
Brainstorm are some ways to find users of illicit substances such as opioids (ages 18+) in the US so that a \
non-profit can provide services to them."""

completion = generate_text(prompt)
Markdown(completion.result)

In [ ]:
Markdown(completion.result)

# Azure OpenAI

In [ ]:
import os
from openai import AzureOpenAI
from IPython.display import Markdown, display 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2023-05-15"
)


In [ ]:
print(os.getenv("AZURE_OPENAI_KEY"))

In [ ]:
response = client.chat.completions.create(
    model="Deployment2", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)


In [ ]:
#openai.api_key = os.getenv("AZURE_OPENAI_KEY")
#openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
#openai.api_type = 'azure'
#openai.api_version = '2023-05-15' # this might change in the future

#deployment_name='Deployment1' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

client=openai.OpenAI()

In [ ]:
def get_response_to_prompt(prompt,model="gpt-4-1106-preview"):
    return client.chat.completions.create(
        model=model,
        temperature=0, 
        messages=[
            # {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
            {"role": "user", "content": prompt}
        ]
    )

#def get_response_to_prompt(prompt):
#    response = openai.ChatCompletion.create(model="gpt-4", \
#               messages=[{"role":"user","content":prompt}], temperature=0)
#    return response.choices[0].message["content"]

In [ ]:
completion = get_response_to_prompt("How to print Hello World") 


In [ ]:
Markdown(completion.choices[0].message.content)


In [ ]:
get_response_to_prompt("Reverse the word lollipop")

In [ ]:
# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'Write a tagline for an ice cream shop. '
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text)


# AWS Bedrock

In [168]:
import boto3
import json


In [169]:
bedrock = boto3.client(service_name='bedrock-runtime',region_name='us-west-2')


In [183]:
body = json.dumps({
    "prompt": "\n\nHuman: explain black holes to 8th graders\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.1,
    "top_p": 0.9,
})

modelId = 'anthropic.claude-v2'
#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'


In [184]:
response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)


In [185]:
print(response)

{'ResponseMetadata': {'RequestId': '9a880264-0ac8-42ef-8222-2e7377b4dc6b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 03 May 2024 05:23:50 GMT', 'content-type': 'application/json', 'content-length': '1445', 'connection': 'keep-alive', 'x-amzn-requestid': '9a880264-0ac8-42ef-8222-2e7377b4dc6b', 'x-amzn-bedrock-invocation-latency': '9970', 'x-amzn-bedrock-output-token-count': '300', 'x-amzn-bedrock-input-token-count': '17'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x122627700>}


In [173]:
temp = response['body'].read()

In [178]:
print(temp)

b'{"completion":" Here is a simple explanation of black holes for 8th graders:\\n\\n- A black hole is a region in space where gravity is so strong that nothing can escape from it, even light. This region is called the \\"event horizon.\\"\\n\\n- Black holes form when very massive stars collapse. When the star runs out of nuclear fuel, gravity makes it collapse. The matter gets compressed into a very small space, creating a black hole. \\n\\n- Black holes are called \\"black\\" because they do not let any light escape. The gravity is so intense that it swallows everything, even light waves. So we can\'t see black holes directly. We can only detect them from their effects on nearby stars and gas.\\n\\n- Black holes have such strong gravity because all their mass is concentrated in a very small space. This creates a very steep gravitational slope near the event horizon that traps everything. \\n\\n- Not even light can escape once it crosses the event horizon. That\'s why it\'s called the 

In [180]:
#response_body = json.loads(response.get('body').read())
response_body = json.loads(temp)

# text
print(response_body.get('completion'))


 Here is a simple explanation of black holes for 8th graders:

- A black hole is a region in space where gravity is so strong that nothing can escape from it, even light. This region is called the "event horizon."

- Black holes form when very massive stars collapse. When the star runs out of nuclear fuel, gravity makes it collapse. The matter gets compressed into a very small space, creating a black hole. 

- Black holes are called "black" because they do not let any light escape. The gravity is so intense that it swallows everything, even light waves. So we can't see black holes directly. We can only detect them from their effects on nearby stars and gas.

- Black holes have such strong gravity because all their mass is concentrated in a very small space. This creates a very steep gravitational slope near the event horizon that traps everything. 

- Not even light can escape once it crosses the event horizon. That's why it's called the "point of no return." Anything that goes beyond 

In [17]:


response_body = response.get('body').read()

# text
response_body

In [29]:
response_body = response.get('body').read()
print(response_body.get('completion'))


AttributeError: 'bytes' object has no attribute 'get'

In [ ]:
print(response_body.get("completions")[0].get("data").get("text"))


In [ ]:
body = json.dumps({
    "prompt": "\n\nHuman: explain black holes to 8th graders\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.0,
    "top_p": 0.9,
})

#modelId = 'anthropic.claude-v2'
modelId = 'ai21.j2-mid-v1'
accept = 'application/json'
contentType = 'application/json'


In [ ]:
response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)


In [ ]:



response_body = json.loads(response.get('body').read())

# text
print(response_body.get('completion'))


# Local Inference Server (not currently working)


In [ ]:
import os
import openai
from IPython.display import Markdown, display 

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']

#client = openai.OpenAI()
#openai.api_base = "http://localhost:1234/v1" # point to the local server
#openai.api_key = "" # no need for an API key


In [ ]:

def get_response_to_prompt(prompt):
    response = openai.ChatCompletion.create(model="gpt-4", \
               messages=[{"role":"user","content":prompt}], temperature=0)
    return response.choices[0].message["content"]

In [ ]:

response = get_response_to_prompt("Who are you")
print(response)

In [ ]:
openai.__version__

In [ ]:
import sys


In [ ]:
sys.executable